In [1]:
import bpy
import numpy as np

import os
from typing import List, Dict
from collections import deque
import bmesh

import scipy.sparse
import scipy.linalg
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from line_profiler import profile

from mathutils import Vector, Matrix
from mathutils.bvhtree import BVHTree

# install packages : "C:\Users\Pierre.Gilibert\OneDrive - ARVERNE\Bureau\blender-4.3.1-windows-x64\4.3\python\bin\python.exe" -m pip install scipy 

In [16]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v01.blend"))

{'FINISHED'}

In [54]:


class HEEdge:
    def __init__(self, edge, face, orientation:int):
        self.edge = edge
        self.face = face
        self.orientation = orientation
        self.vertex = edge.verts[orientation] # orientation in {0,1} stating whether the self.edge is in the same (0) direction as self or the opposite (1) direction.
        self.next = None
        self.twin = None
        self.angle_from_X = 0.0
        self.transport_coeff = 0.0

    def __str__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"
    
    def __repr__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"

    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.edge, item)

    


In [55]:

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2facecorner = {}
        self.facecorner_attributes = {}             # facecorners attributes
        self.vertex_attributes = {"u":{}, "v":{}}   # u,v : basis vector of the tangent plane at each vertex
        self.edge_attributes = {}
        self.face_attributes = {}
        self.face_vertex_to_facecorner = {}
        self.co = None                              # A (|V|, 3) np array where row i contains the x,y,z coordinates of the vertex indexed by i 
        self.cotan = None                           # A (3|F|,) np array where entry i is the cotangent of the angle of facecorner (loop) i
        self.face_areas = None                      # A (|F|,) np array where entry i is the area for face i
        self.internal_angles = None                 # A (3|F|,) np array where entry i is the angle of facecorner (loop) i
        self.fv = None                              # A (|F|, 3) np array where row i contains the indices of the vertices of face i
        self.heedges = []                           # List of Half edges
        self.dict_vert2heedges = {}

    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2facecorner = {v:[] for v in self.verts}
        self.facecorners = []
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2facecorner[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.face_vertex_to_facecorner[face][loop.vert] = loop
            
        self.fv = np.array([[fc.vert.index for fc in f.loops] for f in self.faces], dtype=int) # face #f has vertex [vi, vj, vk]
        self.co = np.array([v.co for v in self.verts])
        
    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Add any other bmesh methods as needed:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()

    def create_halfedge_datastructure(self):
        self.dict_vert2heedges = {}
        self.vertex_attributes["hedge"] = {}
        # self.edge_attributes["heedges"] = {}
        # for e in self.edges:
        #     self.edge_attributes["heedges"][e] = []
        for f in self.faces:
            if len(f.verts) != 3:
                raise ValueError("Not a triangular mesh : triangulate it beforehand !!")
            v_orientation = [f.verts[0].index, f.verts[1].index, f.verts[2].index]
            face_heedges = []
            for e in f.edges:
                # e0, e1, e2 = f.edges[0], f.edges[1], f.edges[2]
                v0, v1 = v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index)
                orientation = 1 - ((abs(v0-v1) == 1 and v0 < v1) or (abs(v0-v1) == 2 and v0 > v1)) # 0 if the edge matches the ccw convention of the face, 1 otherwise
                # print(e, v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index), orientation)
                face_heedges.append(HEEdge(e, f, orientation))
            he0, he1, he2 = face_heedges
            he0.next = he1
            he1.next = he2
            he2.next = he0
            self.dict_vert2heedges[(he0.vertex.index, he1.vertex.index)] = he0
            self.dict_vert2heedges[(he1.vertex.index, he2.vertex.index)] = he1
            self.dict_vert2heedges[(he2.vertex.index, he0.vertex.index)] = he2
            self.heedges.append(he0)
            self.heedges.append(he1)
            self.heedges.append(he2)
            self.vertex_attributes["hedge"][he0.vertex] = he0
            self.vertex_attributes["hedge"][he1.vertex] = he1
            self.vertex_attributes["hedge"][he2.vertex] = he2
            # self.edge_attributes["heedges"][he0.edge] = he0
            # self.edge_attributes["heedges"][he1.edge] = he1
            # self.edge_attributes["heedges"][he2.edge] = he2

        for k in self.dict_vert2heedges:
            i0, i1 = k
            if i0 < i1:
                continue
            kr = (i1, i0)
            if kr in self.dict_vert2heedges:
                self.dict_vert2heedges[k].twin = self.dict_vert2heedges[kr]
                self.dict_vert2heedges[kr].twin = self.dict_vert2heedges[k]
        
        boundary_hedges = []
        for e in self.edges:
            if not e.is_boundary:
                continue
            i0, i1 = e.verts[0].index, e.verts[1].index
            if (i0, i1) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i0, i1)]
            elif (i1, i0) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i1, i0)]
            else:
                raise ValueError(f"Unable to find an half edge between vertices {i0} and {i1}")
            new_he = HEEdge(e, None, 1-existing_he.orientation)
            existing_he.twin = new_he
            new_he.twin = existing_he
            boundary_hedges.append(new_he)
            self.dict_vert2heedges[(new_he.vertex.index, e.other_vert(new_he.vertex).index)] = new_he
            self.heedges.append(new_he)

        # match the boundary hedges
        num_link_to_create = len(boundary_hedges)
        num_link_created = 0
        while num_link_created != num_link_to_create:
            he = boundary_hedges[num_link_created]
            # other_index = -1
            for i, ohe in enumerate(boundary_hedges):
                if ohe.vertex == he.twin.vertex:
                    other_index = i
                    break
            # ohe = boundary_hedges.pop(other_index)
            he.next = ohe
            num_link_created+=1


        # verification : all hedges should have a twin and a next defined
        for k in self.dict_vert2heedges:
            he = self.dict_vert2heedges[k]
            if he.twin is None or he.next is None:
                raise ValueError(f"Unable to define a twin or a next for hedge {k} : {he}")

    def _build_d0(self):

        row = np.repeat(np.arange(len(self.edges)), 2)
        col = np.array([y for e in self.edges for y in [e.verts[0].index, e.verts[1].index]])
        val = np.ones(len(col))
        val[1::2] = -1
        # row, col, val = [], [], []
        # for e in self.edges:
        #     row.append(e.index)
        #     row.append(e.index)
        #     vi, vj = e.verts[0].index, e.verts[1].index
        #     col.append(vi)
        #     col.append(vj)
        #     if vi < vj:
        #         val.append(-1)
        #         val.append(1)
        #     else:
        #         val.append(1)
        #         val.append(-1)

        # matrix[e] = [0,0,..., 1, 0, ..., -1, ..., 0] +1 at vertex vi, -1 at vertex vj where e = [vi, vj] in that order
        return scipy.sparse.coo_matrix((val, (row, col)), shape=(len(self.edges), len(self.verts))).tocsr()

    def _build_d1(self):
        edge_dict = {tuple(sorted([e.verts[0].index, e.verts[1].index])): i for i, e in enumerate(self.edges)}
        vi, vj, vk = self.fv[:,0], self.fv[:,1], self.fv[:,2]
        face_edges = np.reshape(np.vstack((vi, vj, vj, vk, vk, vi)).T, (len(self.faces), 3, 2))
        # face_edges[f] = [[i,j], [j,k], [k, i]] (shape 3,2) with i,j,k the vertex indices of face f and [i,j], [j,k] and [k,i] the 3 ordered edges of face f
        sorted_face_edges = np.sort(face_edges, axis=2)
        val = np.any(sorted_face_edges == face_edges, axis=2)*2-1 # +1 if edge in correct orientation, -1 otherwise
        col = np.array([edge_dict[tuple(e)] for e in sorted_face_edges.reshape((-1,2))])
        row = np.repeat(np.arange(len(self.faces)), 3)
        # matrix[f] = [0, 0,... , +-1, ..., +-1, ..., +-1, ..., 0] non 0 at ei, ej, ek where f.edges = [ei, ej, ek] and +-1 depending on whether the vertices of the ei, ej, ek are in the sorted order or not 
        return scipy.sparse.coo_matrix((val.ravel(), (row, col)), shape=(len(self.faces), len(self.edges))).tocsr()

    def _build_hodge0(self, inverse=False):
        if not "area" in self.vertex_attributes:
            self._calculate_vertex_area()
        vertex_areas = self.vertex_attributes["area"]
        N = len(self.verts)
        row = np.arange(N)
        if inverse:
            return scipy.sparse.coo_matrix((1/vertex_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((vertex_areas, (row, row)), shape=(N, N)) # dual / primal with primal = area of vertex = 1 by convention

    def _build_hodge1(self, inverse=False):
        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_face_areas = np.zeros(len(self.edges))
        for e in self.edges:
            s = 0
            for f in e.link_faces:
                s += self.face_areas[f.index]
            edge_face_areas[e.index] = s

        edge_len = np.linalg.norm(self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]], axis=1)
        # dual_edge_lengths = edge_face_areas / edge_len

        N = len(self.edges)
        row = np.arange(N)
        # return edge_len / edge_face_areas
        if inverse:
            return scipy.sparse.coo_matrix((edge_len / edge_face_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((edge_face_areas / edge_len, (row, row)), shape=(N, N))

    def _build_hodge2(self, inverse=False):
        if not "area" in self.vertex_attributes:
            self._calculate_vertex_area()

        N = len(self.faces)
        row = np.arange(N)
        primal_areas = self.face_areas

        dual_areas = 1 #

        # fc_area = np.array([[self.facecorner_attributes["area"][self.face_vertex_to_facecorner[f][v].index] for f in self.faces for v in f.verts]]).reshape((-1,3))
        # weights = fc_area / self.face_areas[:,None]
        # vertex_area = self.vertex_attributes["area"][self.fv.ravel()].reshape((-1,3))
        # dual_areas = weights*vertex_area
        # dual_areas = dual_areas[:,0] + dual_areas[:,1] + dual_areas[:,2]
        if inverse:
            return scipy.sparse.coo_matrix((primal_areas/dual_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((dual_areas/primal_areas, (row, row)), shape=(N, N))

    def construct_dec_operators(self, inverse_hodge0=False, inverse_hodge1=False, inverse_hodge2=False):
        """
        Constructs discrete exterior derivative operators (d0, d1) and
        Hodge star matrices (hodge0, hodge1, hodge2).
        """
        d0 = self._build_d0()
        d1 = self._build_d1()
        hodge0 = self._build_hodge0(inverse=inverse_hodge0)
        hodge1 = self._build_hodge1(inverse=inverse_hodge1)
        hodge2 = self._build_hodge2(inverse=inverse_hodge2)
        
        return d0, d1, hodge0, hodge1, hodge2

    def vector_field_to_1form(self, vector_field):
        """
        Projects a vertex-based tangent vector field onto a discrete 1-form on edges.

        Parameters:
        - vector_field: Nx3 numpy array, the tangent vector field at each vertex.

        Returns:
        - one_form: Mx1 numpy array, the discrete 1-form (edge-based representation).
        """
        
        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]]
        ui = vector_field[edge_vertex[:,0]]
        uj = vector_field[edge_vertex[:,1]]
        u_edge = 0.5 * (ui+uj)
        dot = np.einsum('ij,ij->i', u_edge, edge_dir) # fast dot product 
        return dot/np.linalg.norm(edge_dir, axis=1)
    
    def one_form_to_vector_field(self, one_form):
        """
        Maps a discrete 1-form (edge-based representation) back to a vertex-based tangent vector field.

        Parameters:
        - one_form: Mx1 numpy array, the discrete 1-form values on edges.

        Returns:
        - vector_field: Nx3 numpy array, the tangent vector field at each vertex.
        """

        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]]
        edge_dir = edge_dir / np.linalg.norm(edge_dir, axis=1)[:,None]
        contribution = one_form[:,None] * edge_dir

        row = np.repeat(np.arange(len(self.edges)),2).astype(int)
        col = edge_vertex.ravel().astype(int)
        val = np.ones(2 * len(self.edges))

        adjacency_matrix = scipy.sparse.coo_matrix((val, (row, col)))

        vertex_contributions = adjacency_matrix.T @ contribution  # Sum contributions per vertex
        vertex_edge_counts = adjacency_matrix.sum(axis=0).A1  # Number of edges per vertex (1D array)

        return vertex_contributions / vertex_edge_counts[:,None]

    def _calculate_corner_area(self):
        if self.internal_angles is None:
            self._calculate_corner_angles_and_face_areas()

        nfaces = len(self.faces)
        ffc = np.arange(3*nfaces).reshape((nfaces, 3)) # face corner index : face #f has corners [i, j, k];
        angles = self.internal_angles[ffc] # array of alpha_i, alpha_j, alpha_k
        eij = self.co[self.fv[:,1]] - self.co[self.fv[:,0]] # edges
        ejk = self.co[self.fv[:,2]] - self.co[self.fv[:,1]]
        eki = self.co[self.fv[:,0]] - self.co[self.fv[:,2]]
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2] # squared lengths of the edges
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]

        # in case triangle i,j,k is non obtuse (all its angles are < pi/2) then this is the area of each face corner
        non_obtuse_area_i = 0.125 * (lij2 * self.cotan[ffc[:,2]] + lki2 * self.cotan[ffc[:,1]])
        non_obtuse_area_j = 0.125 * (ljk2 * self.cotan[ffc[:,0]] + lij2 * self.cotan[ffc[:,2]])
        non_obtuse_area_k = 0.125 * (lki2 * self.cotan[ffc[:,1]] + ljk2 * self.cotan[ffc[:,0]])

        facecorners_areas = np.zeros(3*nfaces).reshape((nfaces, 3))
        # check whether the angles are less than pi/2
        small_angle_bool = angles < np.pi/2
        big_angle_bool = ~small_angle_bool
        # True for all corners of triangle i,j,k ==> use the non_obtuse_area
        non_obtuse_bool = np.logical_and(small_angle_bool[:,0], np.logical_and(small_angle_bool[:,1], small_angle_bool[:,2]))
        facecorners_areas[non_obtuse_bool, 0] = non_obtuse_area_i[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 1] = non_obtuse_area_j[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 2] = non_obtuse_area_k[non_obtuse_bool]
        # False for corner p in {i,j,k}: use half the face area for p and a quarter for the two others corners
        facecorners_areas[big_angle_bool[:,0]] = self.face_areas[big_angle_bool[:,0]][:,None] * np.array([[0.5, 0.25, 0.25]])
        facecorners_areas[big_angle_bool[:,1]] = self.face_areas[big_angle_bool[:,1]][:,None] * np.array([[0.25, 0.5, 0.25]])
        facecorners_areas[big_angle_bool[:,2]] = self.face_areas[big_angle_bool[:,2]][:,None] * np.array([[0.25, 0.25, 0.5]])

        self.facecorner_attributes["area"] = facecorners_areas.ravel()

    def _calculate_corner_angles_and_face_areas(self):

        vi = self.co[self.fv[:,0]]
        vj = self.co[self.fv[:,1]]
        vk = self.co[self.fv[:,2]]

        eij, ejk, eki = vj-vi, vk-vj, vi-vk
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2]
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]
        lij, ljk, lki = np.sqrt(lij2), np.sqrt(ljk2), np.sqrt(lki2)

        s = 0.5 * (lij + ljk + lki) # half perimeter of every triangle
        self.face_areas = np.sqrt(s * (s - lij) * (s - ljk) * (s - lki)) # Heron's formula for the area of the triangles
        
        q_i = -ljk2 + lij2 + lki2
        q_j = -lki2 + ljk2 + lij2
        q_k = -lij2 + lki2 + ljk2

        denom_inv = 1/(4*self.face_areas)
        self.cotan = np.zeros(3*len(bm.faces))
        self.cotan[0::3] = q_i*denom_inv
        self.cotan[1::3] = q_j*denom_inv
        self.cotan[2::3] = q_k*denom_inv

        self.internal_angles = np.zeros(3*len(bm.faces))
        self.internal_angles[0::3] = np.arccos(np.clip(q_i / (2*lij * lki), -1, 1))
        self.internal_angles[1::3] = np.arccos(np.clip(q_j / (2*ljk * lij), -1, 1))
        self.internal_angles[2::3] = np.arccos(np.clip(q_k / (2*lki * ljk), -1, 1))

    def _calculate_vertex_area(self, force_recompute=False):
        # self.vertex_attributes["area"] = np.array([np.sum([self.facecorner_attributes["area"][fc.index] for fc in self.vert2facecorner[v]]) for v in self.verts])
        if not "area" in self.facecorner_attributes or force_recompute:
            self._calculate_corner_area()
            
        val = []
        for v in self.verts:
            s = 0
            for fc in self.vert2facecorner[v]:
                s += self.facecorner_attributes["area"][fc.index]
            val.append(s)
        self.vertex_attributes["area"] = np.array(val)
      
    def assign_distinguished_vector_X(self):
        all_X = []
        for v in self.verts:
            if v.is_boundary:
                boundary_edges = [e for e in v.link_edges if e.is_boundary]
                if len(boundary_edges) != 2:
                    raise ValueError(f"Non manifold mesh : vertex {v.index} does not have exactly 2 adjacent boundary edges")
                other_vs = [boundary_edges[0].other_vert(v), boundary_edges[1].other_vert(v)]
                # select the hedge starting at v and having a non None face
                vector_X = None
                for ov in other_vs:
                    key = (v.index, ov.index)
                    key_r = (ov.index, v.index)
                    for k in [key, key_r]:
                        he = self.dict_vert2heedges[k]
                        if he.vertex == v and he.face is not None:
                            vector_X = he
                            break
                    if vector_X is not None:
                        break
                if vector_X is None:
                    raise ValueError(f"Unable to find an X for boundary vertex {v.index}")
                all_X.append(vector_X)
                # if (v.index, boundary_edges[0].other_vert(v).index) in self.dict_vert2heedges:
                #     all_X.append(self.dict_vert2heedges[(v.index, boundary_edges[0].other_vert(v).index)])
                # elif (v.index, boundary_edges[1].other_vert(v).index) in self.dict_vert2heedges:
                #     all_X.append(self.dict_vert2heedges[(v.index, boundary_edges[1].other_vert(v).index)])
                # else:
                #     raise ValueError(f"Unable to find an X for boundary vertex {v.index}")
            else:
                all_X.append(self.dict_vert2heedges[(v.index, v.link_edges[0].other_vert(v).index)])
        # all_X = np.array(all_X)
        self.vertex_attributes["X"] = all_X 

    def calculate_angle_sum(self, force_recompute=False):
        if self.internal_angles is None or force_recompute:
            self._calculate_corner_angles_and_face_areas()
        angle_sum = np.zeros(len(self.verts))
        for i, v in enumerate(self.verts):
            s = 0
            for fc in self.vert2facecorner[v]:
                s += self.internal_angles[fc.index]
            angle_sum[i] = s

        self.vertex_attributes["angle_sum"] = angle_sum

    def calculate_angle_defect(self, force_recompute=False):
        if not "angle_sum" in self.vertex_attributes or force_recompute:
            self.calculate_angle_sum(force_recompute)

        boundary_coeff =  2 - np.array([v.is_boundary for v in self.verts]) # v on interior : 2 ; v on boundary : 1
        self.vertex_attributes["angle_defect"] = np.pi * boundary_coeff - self.vertex_attributes["angle_sum"]

    def calculate_gaussian_curvature(self, force_recompute=False):
        if not "area" in self.vertex_attributes or force_recompute:
            self._calculate_vertex_area(force_recompute)
        if not "angle_defect" in self.vertex_attributes or force_recompute:
            self.calculate_angle_defect(force_recompute)

        self.vertex_attributes["gaussian_curvature"] = self.vertex_attributes["angle_defect"] / self.vertex_attributes["area"]

    def compute_face_frame(self):

        vi, vj, vk = self.co[self.fv[:,0]], self.co[self.fv[:,1]], self.co[self.fv[:,2]]
        e1 = vj - vi
        e1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        e2 = vk - vi
        e2 = e2 - e1 * np.einsum('ij,ij->i',e1, e2)[:,None]
        e2 = e2/np.linalg.norm(e2, axis=1)[:,None]

        self.face_attributes["e1"] = e1
        self.face_attributes["e2"] = e2

    def shared_halfedge(self, f, g):
        vi, vj = f.verts[0].index, f.verts[1].index
        he:HEEdge
        he = self.dict_vert2heedges[(vi, vj)]
        keep_looping = True
        while keep_looping:
            if he.twin.face == g:
                return he
            he = he.next
            keep_looping = he != self.dict_vert2heedges[(vi,vj)]
        raise ValueError(f"Unable to find a common halfedge between faces {f.index} and {g.index}")
      



In [56]:
obj = bpy.data.objects["Sphere"]
# obj = bpy.data.objects["Sphere.001"]
# obj = bpy.data.objects["Suzanne"]
# obj = bpy.data.objects["Torus.001"]

bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data
bm = MYMesh()
bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.create_halfedge_datastructure()
bm._calculate_corner_angles_and_face_areas()
bm.compute_face_frame()


In [57]:
def calculate_holonomy(self:MYMesh, vertex_index):
    self.compute_face_frame()
    he = self.vertex_attributes["hedge"][self.verts[vertex_index]]
    keep_looping = True
    sum = 0
    counter = 0
    while keep_looping:
        faceI, faceJ = he.face, he.twin.face
        if not (faceI is None or faceJ is None):
            u, v = he.vertex, he.twin.vertex
            e = v.co - u.co
            if u.index > v.index:
                e = -e

            ei1, ei2 = self.face_attributes["e1"][faceI.index], self.face_attributes["e2"][faceI.index]
            ej1, ej2 = self.face_attributes["e1"][faceJ.index], self.face_attributes["e2"][faceJ.index]
            delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
            delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))
            sum += delta_ji - delta_ij
            # print(u.index, v.index, delta_ji, delta_ij)
        he = he.twin.next
        keep_looping = he != self.vertex_attributes["hedge"][self.verts[vertex_index]]
    while sum >= np.pi : sum-=2*np.pi
    while sum < -np.pi : sum+=2*np.pi
    return sum


calculate_holonomy(bm, 4)


-0.009407612598747228

In [58]:

class MyBezier:
    def __init__(self, spline):
        self.spline = spline


    def _retrieve_segment_and_local_coordinate_at(self, t):
        # Ensure t is within [0, 1]
        t = max(0.0, min(1.0, t))

        # Total number of segments (n - 1 control points)
        n_segments = len(self.spline.bezier_points) - 1
        if n_segments < 1:
            raise ValueError("The spline must have at least two bezier points.")

        # Determine which segment the parameter t falls into
        segment_index = int(t * n_segments)
        segment_index = min(segment_index, n_segments - 1)  # Clamp to valid index

        # Compute local parameter s in the segment
        s = (t - segment_index / n_segments) * n_segments

        # Retrieve the control points for the segment
        pA = self.spline.bezier_points[segment_index].co
        hA = self.spline.bezier_points[segment_index].handle_right
        pB = self.spline.bezier_points[segment_index + 1].co
        hB = self.spline.bezier_points[segment_index + 1].handle_left
        return s, pA, hA, pB, hB


    def evaluate(self, t):
        """
        Evaluate the position on the Bezier curve at parameter t (0 <= t <= 1).

        Parameters:
        t (float): A parameter between 0 and 1.

        Returns:
        Vector: The 3D coordinate of the point at t.
        """
        s, pA, hA, pB, hB = self._retrieve_segment_and_local_coordinate_at(t)

        # Compute p(s) using the cubic Bezier formula
        p_s = ((1 - s) ** 3) * pA + \
              3 * ((1 - s) ** 2) * s * hA + \
              3 * (1 - s) * (s ** 2) * hB + \
              (s ** 3) * pB

        return p_s


    def evaluate_derivative(self, t):
        """
        Compute the tangent (derivative) of the Bezier curve at parameter t.

        Parameters:
        t (float): A parameter between 0 and 1.

        Returns:
        Vector: The 3D tangent vector at t.
        """
        s, pA, hA, pB, hB = self._retrieve_segment_and_local_coordinate_at(t)
        p_prime_s = -3 * ((1 - s) ** 2) * pA + \
                     3 * ((1 - s) ** 2 - 2 * (1 - s) * s) * hA + \
                     3 * (2 * (1 - s) * s - s ** 2) * hB + \
                     3 * (s ** 2) * pB
        return p_prime_s.normalized()


class MyNurbs:
    def __init__(self, spline):
        self.spline = spline

    def evaluate(self, t):
        """Evaluate the NURBS curve at parameter t (0 <= t <= 1)."""
        points = [p.co for p in self.spline.points]
        weights = [p.weight for p in self.spline.points]
        n = len(points) - 1

        # Calculate the basis functions (B-splines)
        def basis(i, k, t):
            if k == 0:
                return 1.0 if i / n <= t <= (i + 1) / n else 0.0
            else:
                left = ((t - i / n) / (k / n)) * basis(i, k - 1, t) if k / n > 0 else 0
                right = (((i + k + 1) / n - t) / (k / n)) * basis(i + 1, k - 1, t) if k / n > 0 else 0
                return left + right

        # Evaluate curve
        numerator = sum(weights[i] * points[i] * basis(i, self.spline.order_u, t) for i in range(n + 1))
        denominator = sum(weights[i] * basis(i, self.spline.order_u, t) for i in range(n + 1))
        return numerator / denominator if denominator != 0 else Vector((0, 0, 0))

    def evaluate_derivative(self, t):
        """Compute the tangent at parameter t."""
        delta_t = 0.001
        p0 = self.evaluate(max(0, t-delta_t))
        p1 = self.evaluate(t)
        p2 = self.evaluate(min(1.0, t + delta_t))
        return (p2 - p1).normalized()


class MyPoly:
    def __init__(self, spline):
        self.spline = spline

    def evaluate(self, t):
        """Evaluate a polyline at parameter t."""
        points = [p.co for p in self.spline.points]
        segment_count = len(points) - 1
        segment_index = min(int(t * segment_count), segment_count - 1)
        segment_t = (t * segment_count) - segment_index
        return (1 - segment_t) * points[segment_index] + segment_t * points[segment_index + 1]

    def evaluate_derivative(self, t):
        """Compute the tangent of a polyline."""
        points = [p.co for p in self.spline.points]
        segment_count = len(points) - 1
        segment_index = min(int(t * segment_count), segment_count - 1)
        return (points[segment_index + 1] - points[segment_index]).normalized()


def wrap_spline(spline):
    """Wrap a spline object in the appropriate class."""
    if spline.type == 'BEZIER':
        return MyBezier(spline)
    elif spline.type == 'NURBS':
        return MyNurbs(spline)
    elif spline.type == 'POLY':
        return MyPoly(spline)
    else:
        raise ValueError(f"Unsupported spline type: {spline.type}")


In [140]:
    
def load_guide_curves(collection_name):
    """
    Load all Bezier curves from the specified collection.

    Parameters:
    collection_name (str): The name of the collection containing the Bezier curves.

    Returns:
    list: A list of curve objects (bpy.types.Curve) representing the Bezier curves.
    """
    curves = []
    
    # Get the collection
    if collection_name in bpy.data.collections:
        collection = bpy.data.collections[collection_name]
        
        # Iterate through objects in the collection
        for obj in collection.objects:
            if obj.type == 'CURVE':             
                curves.append(obj)
    else:
        print(f"Collection '{collection_name}' not found.")
    
    return curves

def curve_2_polyline(curve, sampling_step):
    """
    Discretize a curve into a polyline by sampling points along its length.

    Parameters:
    curve (bpy.types.Object): The curve object to discretize.
    sampling_step (float): The distance between sampled points.

    Returns:
    list: A list of tuples, each containing (point, tangent), where:
          - point (Vector): The position of the sampled point.
          - tangent (Vector): The tangent at the sampled point.
    """
    polyline = []

    if curve.type != 'CURVE':
        raise ValueError("Provided object is not a curve.")

    # Iterate through splines in the curve
    for spline in curve.data.splines:
        # print(spline.bezier_points)
        for ii, p in enumerate(spline.bezier_points):
            draw_sphere(p.co, 0.005, f"bezier_{ii}")
            # print(p.handle_left, p.handle_right, p.co)
        wrapped_spline = wrap_spline(spline)

        # Estimate the number of points based on spline length
        total_length = spline.calc_length()  # Length of the spline
        num_points = max(2, int(total_length / sampling_step))  # Number of points

        # Generate sampled points along the curve
        for i in range(num_points):
            t = i / (num_points - 1)  # Parameter along the curve (0 to 1)
            point = wrapped_spline.evaluate(t)
            # tangent = wrapped_spline.evaluate_derivative(t)

            # Transform point and tangent to world coordinates
            point_world = curve.matrix_world @ point
            # tangent_world = curve.matrix_world.to_3x3() @ tangent

            polyline.append(point_world)

    return polyline

def polyline_to_mesh(polyline_pts, tangents, name="PolylineMesh"):
    """
    Create a Blender mesh object from a polyline.

    Parameters:
    polyline_pts (list of [Vector]): A list of [point] where:
    tangents (list of [Vector]): A list of [tangent] where:
                                         - point (Vector): A 3D coordinate.
                                         - tangent (Vector): A 3D tangent vector 
    name (str): The name of the created mesh object.

    Returns:
    bpy.types.Object: The created mesh object.
    """
    if not polyline_pts or len(polyline_pts) < 2:
        raise ValueError("Polyline must contain at least two points.")

    # Extract points from the polyline
    # points = [p[0] for p in polyline]
    # tangents = [p[1] for p in polyline]

    # Create a new mesh and object
    mesh = bpy.data.meshes.new(name)
    obj = bpy.data.objects.new(name, mesh)

    # Link the object to the current collection
    bpy.context.collection.objects.link(obj)

    # Create the mesh data
    mesh.from_pydata(polyline_pts, [(i, i + 1) for i in range(len(polyline_pts) - 1)], [])

    if 'tangent_at_p' in mesh.attributes:
        mesh.attributes.remove(mesh.attributes["tangent_at_p"])

    attr = mesh.attributes.new(name="tangent_at_p", type='FLOAT_VECTOR', domain='POINT')
    attr.data.foreach_set('vector', np.array(tangents).flatten())

    # Update the mesh to ensure it displays correctly in Blender
    mesh.update()
    
    return obj

def ensure_helpers_collection():
    """
    Ensure that a collection named "Helpers" exists in the scene.
    Returns the "Helpers" collection.
    """
    if "Helpers" not in bpy.data.collections:
        helpers_collection = bpy.data.collections.new("Helpers")
        bpy.context.scene.collection.children.link(helpers_collection)
    return bpy.data.collections["Helpers"]


def draw_cylinder(pstart, pend, radius, name="Helper_Cylinder"):
    """
    Draw a cylinder using Blender's bmesh module.

    Parameters:
    pstart (Vector): The center of the bottom circular face.
    pend (Vector): A point on the center of the top circular face.
    radius (float): The radius of the cylinder.

    Returns:
    The Blender object representing the cylinder.
    """
    # Delete all objects named "Cylinder" in the scene
    for obj in bpy.data.objects:
        if obj.name == name:
            bpy.data.objects.remove(obj, do_unlink=True)
    # Calculate the cylinder's length (height) and direction vector
    direction = pend - pstart
    height = direction.length

    # Normalize the direction vector
    direction.normalize()

    # Create a new mesh and object for the cylinder
    mesh = bpy.data.meshes.new(name)
    obj = bpy.data.objects.new(name, mesh)
    # bpy.context.collection.objects.link(obj)
    # Add the object to the "Helpers" collection
    helpers_collection = ensure_helpers_collection()
    helpers_collection.objects.link(obj)

    # Create a bmesh object to define geometry
    bm = bmesh.new()
    bmesh.ops.create_cone(
        bm,
        cap_ends=True,
        segments=16,
        radius1=radius,
        radius2=radius/4,
        depth=height
    )

    # Apply a transformation matrix to orient the cylinder
    # Align the cylinder along the specified axis
    up = Vector((0, 0, 1))
    rot_axis = up.cross(direction)
    if rot_axis.length > 0:  # Ensure the rotation axis is valid
        rot_axis.normalize()
        angle = up.angle(direction)
        rot_matrix = Matrix.Rotation(angle, 4, rot_axis)

        bmesh.ops.rotate(bm, verts=bm.verts, cent=(0, 0, 0), matrix=rot_matrix)

    # Translate the cylinder to the starting position
    translation = bmesh.ops.translate(bm, verts=bm.verts, vec=(pstart+pend)/2)

    # Write the bmesh data to the mesh
    bm.to_mesh(mesh)
    bm.free()


def draw_sphere(centre, radius, name):
    """
    Draw a sphere using Blender's bmesh module.

    Parameters:
    centre (Vector): The center of the sphere.
    radius (float): The radius of the sphere.

    Returns:
    The Blender object representing the sphere.
    """
    # Delete all objects named "Sphere" in the scene
    for obj in bpy.data.objects:
        if obj.name == name:
            bpy.data.objects.remove(obj, do_unlink=True)

    # Create a new mesh and object for the sphere
    mesh = bpy.data.meshes.new(name)
    obj = bpy.data.objects.new(name, mesh)
    # bpy.context.collection.objects.link(obj)
    # Add the object to the "Helpers" collection
    helpers_collection = ensure_helpers_collection()
    helpers_collection.objects.link(obj)

    # Create a bmesh object to define geometry
    bm = bmesh.new()
    bmesh.ops.create_uvsphere(
        bm,
        u_segments=4,
        v_segments=4,
        radius= radius
    )

    # Translate the sphere to the specified center
    bmesh.ops.translate(bm, verts=bm.verts, vec=centre)

    # Write the bmesh data to the mesh
    bm.to_mesh(mesh)
    bm.free()

    return obj




def project_polyline_to_mesh(polyline, target_bmesh):
    """
    Project a polyline onto the surface of a given mesh.

    Parameters:
    polyline (list of Vector): The input polyline (a list of 3D points).
    target_bmesh (bmesh): The mesh object onto which the polyline will be projected.

    Returns:
    list of Vector: The projected polyline points.
    """
    # Ensure the target object is a mesh
    # if target_bmesh.type != 'MESH':
    #     raise ValueError("Target mesh must be of type 'MESH'.")

    # Create a BVH tree for the target mesh
    # mesh_data = target_mesh.data
    # bm = bmesh.new()
    # bm.from_mesh(mesh_data)
    target_bmesh.normal_update()
    bvh_tree = BVHTree.FromBMesh(target_bmesh)
    # bm.free()

    # Project each polyline point onto the mesh
    projected_polyline = []
    face_indices = []
    for point in polyline:
        # Find the nearest point on the mesh surface
        location, _, index, _ = bvh_tree.find_nearest(point)
        if location is not None:
            projected_polyline.append(location)
            face_indices.append(index)
        else:
            # If no projection is found (rare case), use the original point
            # projected_polyline.append(point)
            pass

    return projected_polyline, face_indices

def resample_polyline_1_point_per_face(polyline, face_indices):
    """
    Resample the polyline to ensure there is only one point per face.

    Parameters:
    polyline (list of Vector): The projected polyline (points on the mesh).
    face_indices (list of int): The indices of the faces the polyline points were projected onto.

    Returns:
    list of Vector: The resampled polyline with one point per face.
    """
    resampled_polyline = []
    visited_faces = []  # Track faces that have already contributed a point
    for i, face_index in enumerate(face_indices):
        if face_index not in visited_faces:
            visited_faces.append(face_index)  # Mark this face as visited
            resampled_polyline.append(polyline[i])  # Add the corresponding point
    return resampled_polyline, visited_faces

def get_tangent(polyline):
    """
    Calculate the tangent vectors for each point in the polyline.

    Parameters:
    polyline (list of Vector): The polyline points.

    Returns:
    list of Vector: The tangent vectors at each point.
    """
    tangents = []

    for i in range(len(polyline)):
        if i == 0:
            # Forward difference for the first point
            tangent = polyline[i + 1] - polyline[i]
        elif i == len(polyline) - 1:
            # Backward difference for the last point
            tangent = polyline[i] - polyline[i - 1]
        else:
            # Central difference for all other points
            tangent = (polyline[i + 1] - polyline[i - 1]) / 2

        tangents.append(tangent.normalized())  # Normalize for unit length

    return tangents

# Load curves from the "GuideCurves" collection
guide_curves = load_guide_curves("GuideCurves")
polylines, tangents, face_indices = [], [], []

# Discretize each curve into polylines
for index_polyline, curve in enumerate(guide_curves):
    sampling_step = 0.05  # Adjust the sampling step as needed
    polyline = curve_2_polyline(curve, sampling_step)
    for obj in bpy.data.objects:
        if obj.name.startswith("polyline_"):
            bpy.data.objects.remove(obj, do_unlink=True)

    # polyline = [p[0] for p in polyline]
    polyline, face_idx = project_polyline_to_mesh(polyline, bm.bm)
    polyline, face_idx = resample_polyline_1_point_per_face(polyline, face_idx)
    tangent = get_tangent(polyline)
    polyline_to_mesh(polyline, tangent, name=f"polyline_{index_polyline}")
    polylines.append(polyline)
    tangents.append(tangent)
    face_indices.append(face_idx)


face_indices = [y for x in face_indices for y in x]
tangents = [y for x in tangents for y in x]

print(f"{len(polylines)} curves loaded")

1 curves loaded


In [141]:
def build_primal_spanning_tree(self:MYMesh):
    """
    Build a spanning tree of primal edges
    """
    # initialise the parent of each vertex to None : it means the vertex has not been visited yet.
    self.vertex_attributes["parent"] = {v:v for v in self.verts}
    has_been_seen = np.zeros(len(self.verts), dtype=bool)
    while not np.all(has_been_seen): # in practice : this loops over mesh islands
        root = None
        for v in self.verts:
            if not v.is_boundary and not has_been_seen[v.index]: # root is an unvisited non boundary vertex
                root = v
                break
        if root is None:
            raise ValueError("Unable to find a non-boundary vertex")
        
        queue = deque()
        queue.append(root)
        he:HEEdge

        while len(queue)>0:
            v = queue.popleft()
            has_been_seen[v.index] = True
            he = self.vertex_attributes["hedge"][v]
            keep_looping = True
            counter = 0
            while keep_looping and counter < len(v.link_edges) + 1:
                w = he.twin.vertex
                has_been_seen[w.index] = True # set to true even if it is on a boundary
                not_visited = self.vertex_attributes["parent"][w] == w
                if not_visited and w != root and not w.is_boundary:
                    self.vertex_attributes["parent"][w] = v
                    queue.append(w)
                he = he.twin.next
                keep_looping = he != self.vertex_attributes["hedge"][v]
                counter += 1
                if counter == len(v.link_edges)+1:
                    raise ValueError(f"Infinite loop detected : {counter} / {len(v.link_edges)+1}")

def in_primal_spanning_tree(self:MYMesh, he:HEEdge):
    v = he.vertex
    w = he.twin.vertex
    return self.vertex_attributes["parent"][v] == w or self.vertex_attributes["parent"][w] == v
    

def in_dual_spanning_tree(self:MYMesh, he:HEEdge):
    f = he.face
    g = he.twin.face
    return self.face_attributes["parent"][g] == f or self.face_attributes["parent"][f] == g

def build_dual_spanning_cotree(self:MYMesh):
    """
    builds a spanning tree of dual edges that do not cross the primal tree
    """
    def is_boundary(face):
        return face.edges[0].is_boundary or face.edges[1].is_boundary or face.edges[2].is_boundary
    
    self.face_attributes["parent"] = {f:f for f in self.faces}
    has_been_seen = np.zeros(len(self.faces), dtype=bool)  

    while not np.all(has_been_seen): # in practice : this loops over mesh islands
        root = None
        for f in self.faces:
            if is_boundary(f) or has_been_seen[f.index]:
                continue
            root = f # take a face that is not boundary and unvisited so far
            break
        

        queue = deque()
        queue.append(root)
        he:HEEdge
        while len(queue)>0:
            f = queue.popleft()
            has_been_seen[f.index] = True
            vi, vj = f.verts[0].index, f.verts[1].index
            he = self.dict_vert2heedges[(vi, vj)]
            keep_looping = True
            while keep_looping:
                g = he.twin.face
                if g is not None:
                    has_been_seen[g.index] = True
                    if self.face_attributes["parent"][g] == g and g != root and not in_primal_spanning_tree(self, he):
                        self.face_attributes["parent"][g] = f
                        queue.append(g)
                he = he.next
                keep_looping = he != self.dict_vert2heedges[(vi, vj)]


def build_tree_cotree_decomposition(self:MYMesh):
    
    build_primal_spanning_tree(self)
    build_dual_spanning_cotree(self)


def get_shared_halfedge(self:MYMesh, faceA, faceB):
    vi, vj = faceA.verts[0].index, faceA.verts[1].index
    he = self.dict_vert2heedges[(vi,vj)]
    keep_looping = True
    counter = 0
    while keep_looping and counter < 1 + len(faceA.edges):
        if he.twin.face == faceB:
            return he
        he = he.next
        counter += 1
    raise ValueError(f"Unable to find a common (he)edge between face {faceA.index} and face {faceB.index}")
    

def get_dual_cycles(self:MYMesh):

    def get_half_cycle(f):
        hcycle = []
        counter = 0
        while counter < len(self.faces)+1:
            f_parent = self.face_attributes['parent'][f]
            if f == f_parent:
                break
            hcycle.append(get_shared_halfedge(self, f, f_parent))
            f = f_parent
            counter += 1
        return hcycle
    
    cycles = []
    for e in self.edges:
        if e.is_boundary:
            continue
        vi, vj = e.verts[0], e.verts[1]
        he = self.dict_vert2heedges[(vi.index, vj.index)]

        if not in_primal_spanning_tree(self, he) and not in_dual_spanning_tree(self, he):
            g = [he]
            c1 = get_half_cycle(he.twin.face)
            c2 = get_half_cycle(he.face)
            # remove common hedge:
            m,n = len(c1)-1, len(c2)-1
            while c1[m] == c2[n]:
                n -= 1
                m -= 1
            # add them to the generator in the correct orientation
            for i in range(0, m+1):
                g.append(c1[i])
            for i in range(n, -1, -1):
                g.append(c2[i].twin)

            # remove hedges having a dangling vertex 
            vertex_occurence = {}
            for he in g:
                vertex_occurence[he.vertex] = 0
                vertex_occurence[he.twin.vertex] = 0
            for he in g:
                vertex_occurence[he.vertex] += 1
                vertex_occurence[he.twin.vertex] += 1
            to_keep = []
            for he in g:
                ocs, oce = vertex_occurence[he.vertex], vertex_occurence[he.twin.vertex]
                if min(ocs, oce) > 1: # not dangling
                    to_keep.append(he)
            # reorder the hedges:
            for i in range(len(to_keep)-1):
                heA, heB = to_keep[i], to_keep[i+1]
                if heA.vertex == heB.vertex:
                    to_keep[i] = heA.twin

            
            if to_keep[0].vertex.is_boundary or to_keep[0].twin.vertex.is_boundary: # boundary loop. handle with caution
                # find an hedge exactly on the boundary
                # print("A")

                  
                # mesh_new = bpy.data.meshes.new("MyMesh")
                # obj_new = bpy.data.objects.new(f"{obj.name}_dual_generators", mesh_new)
                # bpy.context.collection.objects.link(obj_new)
                # tmp_mesh = bmesh.new()

                # for v in bm.verts:
                #     tmp_mesh.verts.new(v.co)
                # tmp_mesh.verts.ensure_lookup_table()
                # for cycle in [to_keep]:
                #     for he in cycle:
                #         vi, vj = he.verts[0], he.verts[1]
                #         vi, vj = tmp_mesh.verts[vi.index], tmp_mesh.verts[vj.index]
                #         try:
                #             tmp_mesh.edges.new((vi, vj))
                #         except ValueError:
                #             pass
                    
                # tmp_mesh.to_mesh(mesh_new)



                he0 = to_keep[0]
                he = he0
                # draw_cylinder(he.vertex.co, he.twin.vertex.co, 0.005, f"edge_{he.edge.index}")
                keep_looping = True
                counter = 0
                while keep_looping:
                    he = he.next
                    # if counter > 180:
                    #     draw_cylinder(he.vertex.co, he.twin.vertex.co, 0.005, f"edge_{he.edge.index}")
                    keep_looping = not(he.face is None or he.twin.face is None)
                    counter += 1
                    if counter > 4:
                        counter = 0
                        he = he0.twin.next
                    
                
                if he.twin.face is None:
                    he = he.twin
                # now recreate the loop by following the emptyness
                vi, vj = he.vertex.index, he.twin.vertex.index
                to_keep = []
                keep_looping = True

                while keep_looping:

                    to_keep.append(he)
                    he = he.next
                    keep_looping = he != self.dict_vert2heedges[(vi, vj)]
                
            g = to_keep
            cycles.append(g)

            
    return cycles

In [142]:

build_tree_cotree_decomposition(bm)

dual_cycles = get_dual_cycles(bm)

In [151]:
def tip_angle(x, a, b):
       # returns the angle between (a-x) and (b-x)
       x, a, b = np.array(x), np.array(a), np.array(b)
       u, v = a-x, b-x
       u, v = u/np.linalg.norm(u), v/np.linalg.norm(v)
       return np.arctan2(np.linalg.norm(np.cross(u,v)), np.dot(u,v))
    

class TrivialConnection():
    
    def __init__(self, mesh:MYMesh):
        self.mesh = mesh
        self.singularities = None

    def read_singularities(self):
        # self.singularities = np.array([[0, 2]]) #one singularity on vertex 0 of index +2
        # self.singularities = np.array([[np.random.randint(len(self.mesh.verts)), 2]]) #one singularity on vertex 0 of index +2
        self.singularities = np.array([[0, 1], [973, 1]]) # two singularities on vertices 0, 300 of index +1
        # self.singularities = np.array([[0, 1], [300, -1], [1500, 2]]) # three singularities on vertices 0, 300, 500 of index +1, -1, +2
        # self.singularities = np.array([[0, 0.5], [300, -.5], [1500, 2]]) # three singularities on vertices 0, 300, 500 of index +.5, -.5, +2
        # self.singularities = np.array([[]])

    def compute_trivial_connection(self, A, K):
        # A = d0.T
        b = K.copy()
        for row in self.singularities:
            vertex_index, singularity_index = row
            vertex_index = int(vertex_index)
            b[vertex_index] = K[vertex_index] - 2 * np.pi * singularity_index

        AT = A.T
        AAT = A @ AT
        y = spsolve(AAT, -b)
        x = AT @ y

        print(f"max holonomy error = {np.max(np.abs(A@x+b))}")

        return x, A

    def parallel_transport(self, w0, faceI, faceJ, he:HEEdge, x):
        """
        Transport unit vector w0 frome faceI to faceJ separated by halfe edge he using the connection specified by x
        """

        ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
        ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
        ei3, ej3 = np.cross(ei1, ei2), np.cross(ej1, ej2)
        EI = np.vstack((ei1, ei2, ei3)).T
        EJ = np.vstack((ej1, ej2, ej3)).T

        k = he.edge.index
        u, v = he.vertex, he.twin.vertex
        e = v.co - u.co
        if u.index > v.index:
            e = -e
        delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
        delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))

        angle = x[k] - delta_ij + delta_ji
        # angle = 0 - delta_ij + delta_ji

        c, s = np.cos(angle), np.sin(angle)
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
        w = EJ @ R @ EI.T @ w0
        return w
    
    def parallel_transport_angle(self, phi:float, he:HEEdge):
#       given an angle phi relative to the canonical reference frame
#       of he.face, returns the angle parallel transported across he
#       using the Levi-Civita connection, expressed relative to the
#       canonical frame of he.twin.face
        u = he.vertex
        v = he.twin.vertex
        e = v.co - u.co
        if u.index > v.index:
            e = -e
        ei1, ei2 = self.mesh.face_attributes["e1"][he.face.index], self.mesh.face_attributes["e2"][he.face.index]
        ej1, ej2 = self.mesh.face_attributes["e1"][he.twin.face.index], self.mesh.face_attributes["e2"][he.twin.face.index]
        delta_ij = np.arctan2(np.dot(e,ei2), np.dot(e,ei1))
        delta_ji = np.arctan2(np.dot(e,ej2), np.dot(e,ej1))
        return phi - delta_ij + delta_ji

    def compute_vector_field(self):
        self.read_singularities()

        d0 = self.mesh._build_d0()
        self.mesh.calculate_gaussian_curvature(force_recompute=True)
        K = self.mesh.vertex_attributes["angle_defect"]
        
        x, A = self.compute_trivial_connection(d0, K)
        vector_field = self.extract_vector_field(x, A)
        return vector_field

    def draw_singularities(self):

        for obj in bpy.data.objects:
            if obj.name.startswith("singularity_") :
                bpy.data.objects.remove(obj, do_unlink=True)
        
        for row in self.singularities:
            if len(row) == 0:
                continue
            vertex_index = int(row[0])
            co = self.mesh.verts[vertex_index].co
            draw_sphere(co, 0.025, f"singularity_{vertex_index}")

    def extract_vector_fieldOLD(self, x, A, vertex2row, transport_order):
        raise NotImplementedError
        vector_field = np.zeros((len(self.mesh.faces), 3))
        alphas = np.zeros(len(self.mesh.faces))
        alphas.fill(np.nan)
        root_face = transport_order[0].face
        if not np.isnan(self.mesh.face_attributes["constrained_angles"][root_face.index]):
            alphas[root_face.index] = self.mesh.face_attributes["constrained_angles"][root_face.index]

        vector_field[root_face.index] = rotate_axis_angle_single_vector(self.mesh.face_attributes["e1"][root_face.index],
                                                                        root_face.normal, 
                                                                        alphas[root_face.index])
        he:HEEdge
        for he in transport_order:
            faceI, faceJ = he.face, he.twin.face
            if np.isnan(alphas[faceI.index]):
                print(faceI)
                err
            
            u, v = he.vertex, he.twin.vertex
            e = v.co - u.co
            
            # sgn = A[he.vertex.index, he.edge.index]
            sgn = A[vertex2row[he.vertex], he.edge.index]
            ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
            ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
            delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
            delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))
            alpha_i = alphas[faceI.index]
            alpha_j = alpha_i - delta_ij + delta_ji - sgn * x[he.edge.index]
            while alpha_j > np.pi : alpha_j-= 2*np.pi
            while alpha_j <= -np.pi : alpha_j+= 2*np.pi
            alphas[faceJ.index] = alpha_j
            vector_field[faceJ.index] = rotate_axis_angle_single_vector(ej1, faceJ.normal, alpha_j)
        return vector_field, alphas



    def extract_vector_field(self, x, A, vertex2row, transport_order):
        BREAK_DEBUG = False
        nfaces = len(self.mesh.faces)
        vector_field = np.zeros((nfaces, 3))
        has_been_visited = np.zeros(nfaces, dtype=bool)
        while not np.all(has_been_visited) and not BREAK_DEBUG:
            # Handling different mesh island (has_been_visited is useless if there is only one mesh island)
            # Find an unvisited constrained face to set as the root. if no face is constrained or all constrained faces
            # have already been visited, then whatever face will work
            for f in self.mesh.faces:
                if not has_been_visited[f.index]:
                    root = f
                    break

            initial_angle = np.pi/2
            # initial_angle = (np.random.rand() -0.5)*2*np.pi

            for f in self.mesh.faces:
                if not np.isnan(self.mesh.face_attributes["constrained_angles"][f.index]) and not has_been_visited[f.index]:
                    root = f
                    initial_angle = self.mesh.face_attributes["constrained_angles"][f.index]
                    break
            
            alphas = np.zeros(nfaces)
            alphas[root.index] = initial_angle

            p0, p1 = root.verts[0].co, root.verts[1].co
            v0 = p1 - p0
            v0 = v0 / np.linalg.norm(v0)

            v0 = rotate_axis_angle_single_vector(v0, root.normal, initial_angle)
            vector_field[root.index] = v0
            has_been_visited[root.index] = True

            # /////////////////////////////////////////////////////////////////////////////////////
            # first rotate all constrained faces:
            queue = deque()
            for f in self.mesh.faces:
                if not np.isnan(self.mesh.face_attributes["constrained_angles"][f.index]):
                    angle = self.mesh.face_attributes["constrained_angles"][f.index]
                    e1 = self.mesh.face_attributes["e1"][f.index]
                    alphas[f.index] = angle
                    vector_field[f.index] = rotate_axis_angle_single_vector(e1, f.normal, angle)
                    has_been_visited[f.index] = True
                    queue.append(f)

            # queue = deque()
            # queue.append(root)
            he:HEEdge
            # A = self.mesh._build_d0().T
            counter = 0
            while len(queue)>0:
                f = queue.popleft()
                vi, vj = f.verts[0].index, f.verts[1].index
                he = self.mesh.dict_vert2heedges[(vi, vj)]
                keep_looping = True
                while keep_looping:
                    faceI, faceJ = he.face, he.twin.face
                    n_neigbours_visited = 0
                    if faceJ is not None and not has_been_visited[faceJ.index]:
                        het = he.twin
                        keep_looping_count = True
                        while keep_looping_count:
                            other_face = het.twin.face
                            if has_been_visited[other_face.index]:
                                n_neigbours_visited+=1
                            het = het.next
                            keep_looping_count = het != he.twin
                            
                        u, v = he.vertex, he.twin.vertex
                        e = v.co - u.co
                        
                        # sgn = A[he.vertex.index, he.edge.index]
                        sgn = A[vertex2row[he.vertex], he.edge.index]
                        ei1, ei2 = self.mesh.face_attributes["e1"][faceI.index], self.mesh.face_attributes["e2"][faceI.index]
                        ej1, ej2 = self.mesh.face_attributes["e1"][faceJ.index], self.mesh.face_attributes["e2"][faceJ.index]
                        delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
                        delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))

                        alpha_i = alphas[faceI.index]
                        alpha_j = alpha_i - delta_ij + delta_ji - sgn * x[he.edge.index]
                        while alpha_j > np.pi : alpha_j-= 2*np.pi
                        while alpha_j <= -np.pi : alpha_j+= 2*np.pi
                        alphas[faceJ.index] = alpha_j

                        vector_field[faceJ.index] = rotate_axis_angle_single_vector(ej1, faceJ.normal, alpha_j)
                        has_been_visited[faceJ.index] = True
                        queue.append(faceJ)
                        # if n_neigbours_visited == 2:
                        #     BREAK_DEBUG = True
                        #     break
                    
                    he = he.next
                    keep_looping = he != self.mesh.dict_vert2heedges[(vi, vj)]
                counter += 1
                # if BREAK_DEBUG or counter == 500:
                #     BREAK_DEBUG = True
                #     break
        # vector_field[faceJ.index] = 2 * vector_field[faceJ.index]
        return vector_field, alphas

    def calculate_constrained_angle(self, face_indices, vectors):
        """
        Calculate the constrained angle of each face in face_indices, such that the constrained direction is given by vectors:
        Args:
            face_indices List[int] : indices of the face being constrained
            vectors List[Vector] : direction in which the field must be constrained
        """
        self.mesh.face_attributes["constrained_angles"] = np.nan * np.ones(len(self.mesh.faces))
        e1 = self.mesh.face_attributes["e1"]
        for i, idx in enumerate(face_indices):
            vec = np.array(vectors[i].normalized())
            face = self.mesh.faces[idx]
            normal = face.normal
            ei1 = e1[idx]
            xprod = np.cross(ei1, vec)
            angle = np.arctan2(np.dot(xprod, normal), np.dot(ei1, vec))
            self.mesh.face_attributes["constrained_angles"][idx] = angle




    def compute_directional_constraint(self):
        transport_order = []
        directional_constraints = []
        holonomies = []
        c_parent = {f:f for f in self.mesh.faces} # point each face to itself to mark that it has not been visited

        transport_root = self.mesh.faces[0]
        for face in self.mesh.faces:
            if not np.isnan(self.mesh.face_attributes["constrained_angles"][face.index]):
                transport_root = face
                break
        he:HEEdge
        queue = deque()
        queue.append(transport_root)
        while len(queue)>0:
            f = queue.popleft()
            vi, vj = f.verts[0].index, f.verts[1].index
            he = self.mesh.dict_vert2heedges[(vi,vj)]
            keep_looping = True
            while keep_looping:
                g = he.twin.face
                if g is not None and c_parent[g] == g and g != transport_root:
                    c_parent[g] = f
                    queue.append(g)
                    transport_order.append(he)

                    if not np.isnan(self.mesh.face_attributes["constrained_angles"][g.index]): # g is constrained
                        # follow this face back up the tree to the most
                        # recent constrained ancestor, adding all the halfedges
                        # in between to a new constraint; also compute the difference
                        # between the two constrained directions relative to transport
                        # via the Levi-Civita connection
                        alpha = self.mesh.face_attributes["constrained_angles"][g.index]
                        constraint = []
                        h = g
                        keep_looping2 = True
                        while keep_looping2:
                            shared_he = self.mesh.shared_halfedge(h, c_parent[h])
                            alpha = self.parallel_transport_angle(alpha, shared_he)
                            constraint.append(shared_he)
                            h = c_parent[h]
                            keep_looping2 = np.isnan(self.mesh.face_attributes["constrained_angles"][h.index]) # stop once you find an already visited constrained face
                        directional_constraints.append(constraint)
                        gamma = self.mesh.face_attributes["constrained_angles"][h.index]
                        holonomy = alpha - gamma
                        while holonomy >= np.pi: holonomy -= 2*np.pi
                        while holonomy < -np.pi: holonomy += 2*np.pi

                        # holonomies.append(np.arccos(np.cos(alpha-gamma))) # acos(cos(alpha)cos(gamma) + sin(alpha)sin(gamma)) (use acos(cos) instead of angle directly to ensure a correct range for the result)
                        holonomies.append(abs(holonomy))
                he = he.next
                keep_looping = he != self.mesh.dict_vert2heedges[(vi,vj)]

        return directional_constraints, holonomies, transport_order

    def compute_1ring_base(self):
        vertex2row, constraints = {}, []
        for v in self.mesh.verts:
            if v.is_boundary:
                vertex2row[v] = -1
                continue

            he = self.mesh.vertex_attributes["hedge"][v]
            cycle = []
            keep_looping = True
            while keep_looping:
                cycle.append(he)
                he = he.twin.next
                keep_looping = he != self.mesh.vertex_attributes["hedge"][v]

            vertex2row[v] = len(constraints)
            constraints.append(cycle)
        return vertex2row, constraints


    def boundary_loop_curvature(self, cycle):
        he:HEEdge
        totalK = 0
        he0 = cycle[0] # he0.face is None by construction
        # virtual vertex on the middle of the empty face
        c = np.zeros(3)
        he = he0
        keep_looping = True
        while keep_looping:
            c = c + np.array(he.vertex.co)
            he = he.next
            keep_looping = he != he0
        c = c / len(cycle)
        # compute the curvature around c
        K = 2 * np.pi
        he = he0
        keep_looping = True
        while keep_looping:
            a = he.vertex.co
            b = he.twin.vertex.co
            K -= tip_angle(c, a, b)
            he = he.next
            keep_looping = he != he0

        totalK += K

            # add the curvature around each of the boundary vertices, using
            # the following labels:
            #    c - virtual center vertex of boundary loop (computed above)
            #    d - current boundary vertex (we walk around the 1-ring of this vertex)
            #    a,b - consecutive interior vertices in 1-ring of d
            #    e,f - boundary vertices adjacent to d
    
        he = he0
        keep_looping = True
        while keep_looping:
            v = he.vertex
            d = v.co
            K = 2 * np.pi
            he2 = self.mesh.vertex_attributes["hedge"][v]
            keep_looping2 = True
            while keep_looping2:
                if he2.face is None:
                    f = he2.next.vertex.co
                    K -= tip_angle(d, f, c)
                else:
                    a = he2.next.vertex.co
                    b = he2.next.next.vertex.co
                    K -= tip_angle(d, a, b)
                    if he2.twin.face is None:
                        e = he2.twin.vertex.co
                        K -= tip_angle(d, c, e)
                he2 = he2.twin.next
                keep_looping2 = he2 != self.mesh.vertex_attributes["hedge"][v]
            totalK += K
            he = he.next
            keep_looping = he != he0

        return totalK


    def get_defect_of_cycle(self, c):
        theta = 0
        he:HEEdge
        for he in c:
            theta = self.parallel_transport_angle(theta, he)
        while theta >= np.pi : theta -= 2*np.pi
        while theta < np.pi : theta += 2*np.pi
        return -theta

    def build_matrix_A(self, cycles):
        row, col, val = [], [], []
        for cycle_index, cycle in enumerate(cycles):
            for he in cycle:
                edge_index = he.edge.index
                vi, vj = he.vertex.index, he.twin.vertex.index
                col.append(edge_index)
                row.append(cycle_index)
                if vi > vj:
                    val.append(-1)
                else:
                    val.append(1)

        return scipy.sparse.coo_matrix((val, (row, col)),shape=(len(cycles), len(self.mesh.edges))).tocsr()

    def build_vector_b(self, vertex2row, dual_cycles, directional_holonomies):

        self.mesh.calculate_gaussian_curvature(force_recompute=True)
        K = self.mesh.vertex_attributes["angle_defect"]
        ring_b = []
        for v in vertex2row:
            if vertex2row[v] < 0 :
                continue
            ring_b.append(K[v.index])

        generator_b = []
        for cycle in dual_cycles:
            if len(cycle) != 0 and cycle[0].vertex.is_boundary: # boundary cycle
                curvature = self.boundary_loop_curvature(cycle)
                generator_b.append(curvature)
            else:
                defect = self.get_defect_of_cycle(cycle)
                generator_b.append(defect)
        print(len(ring_b), len(generator_b), len(directional_holonomies))
        b = np.vstack((
            np.array(ring_b).reshape((-1,1)), 
            np.array(generator_b).reshape((-1,1)), 
            np.array(directional_holonomies).reshape((-1,1))
            ))

        for row in self.singularities:
            if len(row) == 0:
                continue
            vertex_index, singularity_index = row
            vertex_index = int(vertex_index)
            b[vertex_index] = K[vertex_index] - 2 * np.pi * singularity_index

        # nverts = len(self.mesh.verts)
        # for v in self.mesh.verts:
        #     b[v.index] = K[v.index] - 2 * np.pi * 2 / nverts

        return b
            

def rotate_axis_angle_single_vector(vector, axis, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    axis = axis/np.linalg.norm(axis)
    dot = np.dot(axis, vector)
    xprod = np.cross(axis, vector)
    return (1-c) * dot * axis + c * vector + s * xprod


tc = TrivialConnection(bm)
tc.read_singularities()

tc.calculate_constrained_angle(face_indices, tangents)


vertextorow, one_ring_base = tc.compute_1ring_base()

directional_constraints, directional_holonomies, transport_order = tc.compute_directional_constraint()

build_tree_cotree_decomposition(bm)

dual_cycles = get_dual_cycles(bm)

cycles = one_ring_base + dual_cycles + directional_constraints

print(len(one_ring_base), len(dual_cycles), len(directional_constraints))


A = tc.build_matrix_A(cycles)
b = tc.build_vector_b(vertextorow, dual_cycles, directional_holonomies)
# b = tc.build_vector_b(vertextorow, dual_cycles, [])


AT = A.T
AAT = A @ AT
y = spsolve(AAT, -b)
x = AT @ y

# x = scipy.sparse.linalg.lsqr(A, -b)[0]
print(f"max holonomy error = {np.max(np.abs(A@x+b))}")

# x, A = tc.compute_trivial_connection(A, K)
vector_field, alphas = tc.extract_vector_field(x, A, vertextorow, transport_order)

# vector_field = tc.compute_vector_field()
tc.draw_singularities()


if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])
    mesh.attributes.remove(mesh.attributes["e1"])
    # mesh.attributes.remove(mesh.attributes["constrained_angles"])


attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', vector_field.flatten())

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', bm.face_attributes["e1"].flatten())

tmp = []
for v in bm.face_attributes["constrained_angles"]:
    if np.isnan(v):
        tmp.append(0)
    else:
        tmp.append(v)
attr = mesh.attributes.new(name="constrained_angles", type='FLOAT', domain='FACE')
attr.data.foreach_set('value', tmp)

7938 0 27
7938 0 27
max holonomy error = 9.217753769644297


TypeError: foreach_set(..) sequence length mismatch given 47616, needed 0

In [124]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

# Number of faces
num_faces = len(bm.faces)

# Initialize sparse matrix A and vector b
A_data, A_row, A_col = [], [], []
b = np.zeros(num_faces)

# Fill A and b for adjacency relationships
for edge in bm.edges:  # Assuming mesh.edges gives adjacent face pairs
    vA, vB = edge.verts[0].index, edge.verts[1].index
    he = bm.dict_vert2heedges[(vA, vB)]
    faceI, faceJ = he.face, he.twin.face
    u, v = he.vertex, he.twin.vertex
    e = v.co - u.co
    # sgn = A[he.vertex.index, he.edge.index]
    ei1, ei2 = bm.face_attributes["e1"][faceI.index], bm.face_attributes["e2"][faceI.index]
    ej1, ej2 = bm.face_attributes["e1"][faceJ.index], bm.face_attributes["e2"][faceJ.index]
    delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
    delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))
    delta_pq = delta_ji - delta_ij
    
    # Append to A
    A_data.extend([1, -1])
    A_row.extend([edge.index, edge.index])
    A_col.extend([faceI.index, faceJ.index])
    
    # Append to b
    b[edge.index] = delta_pq

constrained_faces = [f.index for f in bm.faces if not np.isnan(bm.face_attributes["constrained_angles"][f.index])]
# Fill A and b for constraints
for j in constrained_faces:
    A_data.append(1)
    A_row.append(len(mesh.edges) + j)
    A_col.append(j)
    b[len(mesh.edges) + j] = 0

# Build sparse matrix
A = sp.coo_matrix((A_data, (A_row, A_col)), shape=(len(mesh.edges) + len(constrained_faces), num_faces))

# Solve the system
x = spsolve(A.T @ A, A.T @ b)


IndexError: index 15872 is out of bounds for axis 0 with size 15872

In [107]:
from scipy.sparse import lil_matrix

def compute_cotangent_weight(he:HEEdge):
    """
    Compute the cotangent weight for an edge shared by two faces.
    
    Args:
        edge: The edge vector or indices of its endpoints.
        face1, face2: The two faces sharing the edge.
        vertices: The vertex positions of the mesh.

    Returns:
        The cotangent weight.
    """
    # face1, face2 = he.face, he.twin.face
    # Identify the opposite vertices in the two faces
    opp_vertex1 = he.next.next.vertex
    opp_vertex2 = he.twin.next.next.vertex
    
    # Compute angles opposite to the edge
    def cotangent_angle(v1, v2, v3):
        u = v1 - v2
        v = v3 - v2
        return np.dot(u, v) / np.linalg.norm(np.cross(u, v))

    cot1 = cotangent_angle(he.vertex.co, opp_vertex1.co, he.twin.vertex.co)
    cot2 = cotangent_angle(he.twin.vertex.co, opp_vertex2.co, he.vertex.co)

    # Return average of cotangents
    return (cot1 + cot2) / 2

n_faces = len(bm.faces)
L = lil_matrix((n_faces, n_faces))
b = np.zeros(n_faces)

for edge in bm.edges:
    vA, vB = edge.verts[0].index, edge.verts[1].index
    he = bm.dict_vert2heedges[(vA, vB)]
    face_i, face_j = he.face, he.twin.face
    weight = compute_cotangent_weight(he)

    # Compute transport angles
    u, v = he.vertex, he.twin.vertex
    e = v.co - u.co
    ei1, ei2 = bm.face_attributes["e1"][face_i.index], bm.face_attributes["e2"][face_i.index]
    ej1, ej2 = bm.face_attributes["e1"][face_j.index], bm.face_attributes["e2"][face_j.index]
    delta_ij = np.arctan2(np.dot(ei2, e), np.dot(ei1, e))
    delta_ji = np.arctan2(np.dot(ej2, e), np.dot(ej1, e))

    # Update Laplacian
    L[face_i.index, face_i.index] += weight
    L[face_j.index, face_j.index] += weight
    L[face_i.index, face_j.index] -= weight
    L[face_j.index, face_i.index] -= weight

    # Update b with transport contribution
    b[face_i.index] -= weight * (delta_ij - delta_ji)
    b[face_j.index] += weight * (delta_ij - delta_ji)


constrained_faces = [f.index for f in bm.faces if not np.isnan(bm.face_attributes["constrained_angles"][f.index])]
# Remove constrained rows/columns
free = np.setdiff1d(np.arange(n_faces), constrained_faces)
L_free = L[free][:, free]
b_free = b[free]


In [113]:
x_free = spsolve(L_free.tocsr(), -b_free)

# Reconstruct full solution
x = np.zeros(n_faces)
x[free] = x_free

In [121]:
new_vf = vector_field.copy()
for f in bm.faces:
    new_vf[f.index] = rotate_axis_angle_single_vector(vector_field[f.index], f.normal, x[f.index])

if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])

attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', new_vf.flatten())



In [104]:
np.sort(b_free)

array([0., 0., 0., ..., 0., 0., 0.])

In [144]:
vf = np.zeros((len(bm.faces), 3))

for f in bm.faces:
    if not np.isnan(bm.face_attributes["constrained_angles"][f.index]):
        angle = bm.face_attributes["constrained_angles"][f.index]
        e1 = bm.face_attributes["e1"][f.index]
        vf[f.index] = rotate_axis_angle_single_vector(e1, f.normal, alphas[f.index])
        vf[f.index] = rotate_axis_angle_single_vector(e1, f.normal, angle)
        print(alphas[f.index], angle)

        
if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])

attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', vf.flatten())



2.683395586248712 2.683395586248712
2.1338904870131414 2.1338904870131414
1.841100956750795 1.841100956750795
-0.010043833796444131 -0.010043833796444131
3.12175130757946 3.12175130757946
0.12108179167303945 0.12108179167303945
-2.910242405216236 -2.910242405216236
-0.9928206572205712 -0.9928206572205712
-1.8488384216228626 -1.8488384216228626
-1.5885862057185594 -1.5885862057185594
-1.5598746989990535 -1.5598746989990535
-0.1499415151377869 -0.1499415151377869
-1.6534347078587759 -1.6534347078587759
-0.1495233568026855 -0.1495233568026855
0.42179812339993306 0.42179812339993306
-0.40739558116758146 -0.40739558116758146
-1.7061254990975194 -1.7061254990975194
-1.9863813360998785 -1.9863813360998785
3.0045751053612135 3.0045751053612135
-3.0420746148238806 -3.0420746148238806
-1.0869222174546835 -1.0869222174546835
-2.000891036297732 -2.000891036297732
-1.3663547334255948 -1.3663547334255948
-0.6135940745489817 -0.6135940745489817
-0.42356428505625215 -0.42356428505625215
1.430166293524

In [149]:


import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as linalg

def solve_constrained_least_squares(A, b, C, d):
    """
    Solve the constrained least squares problem:
        min ||Ax + b||^2 subject to Cx = -d
    using sparse solvers from scipy.sparse.linalg.
    """
    # Formulate the augmented system to solve for the Lagrange multiplier
    # Solve the following system:
    # [ A^T A   C^T ]
    # [ C        0 ]  * [ x ] = [ -A^T b ]
    #                   [ λ ]   [  -d    ]

    # Size of x
    n = A.shape[1]
    m = C.shape[0]

    # Build the augmented matrix [ A^T A   C^T ]
    #                            [ C        0 ]
    A_T_A = A.T @ A + 1e-9 * scipy.sparse.eye(A.shape[1])
    C_T = C.T
    C_0 = sp.csr_matrix((m, m))  # Zero matrix of size m x m
    print(A_T_A.shape, C_T.shape, C.shape, C_0.shape)
    # Assemble the augmented matrix
    augmented_matrix = sp.bmat([[A_T_A, C_T], [C, C_0]], format="csr")

    
    # Right-hand side vector [-A^T b, -d]
    rhs = np.concatenate([-A.T @ b, -d])

    # Solve the system [A^T A  C^T] [x] = [-A^T b]
    #                    [C      0 ] [λ] = [-d]
    # We can now solve the augmented system using a sparse linear solver.
    print(augmented_matrix.shape, rhs.shape)
    solution = linalg.spsolve(augmented_matrix, rhs)

    # Extract the solution for x (primal variables)
    x = solution[:n]

    return x

# Example usage:
# Assuming A, b, C, d are already defined as sparse matrices or numpy arrays
# A = sp.csr_matrix(...)  # Sparse matrix for A
# b = np.array(...)       # Vector b
# C = sp.csr_matrix(...)  # Sparse matrix for C
# d = np.array(...)       # Vector d

# Solve the constrained least squares problem








tc = TrivialConnection(bm)
tc.read_singularities()

tc.calculate_constrained_angle(face_indices, tangents)


vertextorow, one_ring_base = tc.compute_1ring_base()

directional_constraints, directional_holonomies, transport_order = tc.compute_directional_constraint()

build_tree_cotree_decomposition(bm)

dual_cycles = get_dual_cycles(bm)

cycles = one_ring_base + dual_cycles# + directional_constraints

print(len(one_ring_base), len(dual_cycles), len(directional_constraints))


A = tc.build_matrix_A(cycles)
# b = tc.build_vector_b(vertextorow, dual_cycles, directional_holonomies)
b = tc.build_vector_b(vertextorow, dual_cycles, [])

C = tc.build_matrix_A(directional_constraints)
d = np.array(directional_holonomies).reshape((-1,1))

print(A.shape, b.shape, C.shape, d.shape)
x = solve_constrained_least_squares(A, b, C, d)

# # AT = A.T
# # AAT = A @ AT
# # y = spsolve(AAT, -b)
# # x = AT @ y

# # x = scipy.sparse.linalg.lsqr(A, -b)[0]
print(f"max holonomy error = {np.max(np.abs(A@x+b))}")

# x, A = tc.compute_trivial_connection(A, K)
vector_field, alphas = tc.extract_vector_field(x, A, vertextorow, transport_order)

# vector_field = tc.compute_vector_field()
tc.draw_singularities()


if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])
    mesh.attributes.remove(mesh.attributes["e1"])
    # mesh.attributes.remove(mesh.attributes["constrained_angles"])


attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', vector_field.flatten())

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', bm.face_attributes["e1"].flatten())

tmp = []
for v in bm.face_attributes["constrained_angles"]:
    if np.isnan(v):
        tmp.append(0)
    else:
        tmp.append(v)
attr = mesh.attributes.new(name="constrained_angles", type='FLOAT', domain='FACE')
attr.data.foreach_set('value', tmp)

7938 0 27
7938 0 0
(7938, 23808) (7938, 1) (27, 23808) (27, 1)
(23808, 23808) (23808, 27) (27, 23808) (27, 27)
(23835, 23835) (23835, 1)
max holonomy error = 6.30121787826025


In [88]:
print(np.max(np.abs(C@x + d.ravel())))
print(np.max(np.abs(A@x + b)))

1.5543122344752192e-15
6.301218120839497


In [14]:

print(f"n_generators = {len(dual_cycles)}")


n_generators = 2


In [15]:

# Create a new mesh and object
mesh_new = bpy.data.meshes.new("MyMesh")
obj_new = bpy.data.objects.new(f"{obj.name}_primal_tree", mesh_new)
bpy.context.collection.objects.link(obj_new)

tmp_mesh = bmesh.new()

for v in bm.verts:
    tmp_mesh.verts.new(v.co)
tmp_mesh.verts.ensure_lookup_table()

for v,w in bm.vertex_attributes["parent"].items():
    if v == w:
        continue
    vn, wn = tmp_mesh.verts[v.index],tmp_mesh.verts[w.index]
    tmp_mesh.edges.new((vn,wn))

tmp_mesh.to_mesh(mesh_new)


# Create a new mesh and object
mesh_new = bpy.data.meshes.new("MyMesh")
obj_new = bpy.data.objects.new(f"{obj.name}_dual_tree", mesh_new)
bpy.context.collection.objects.link(obj_new)
tmp_mesh = bmesh.new()
dict_tmp = {}
for f in bm.faces:
    vi, vj, vk = f.verts[0], f.verts[1], f.verts[2]
    barycentre = (vi.co + vj.co + vk.co)/3
    tmp_mesh.verts.new(barycentre)
    dict_tmp[f.index] = barycentre
tmp_mesh.verts.ensure_lookup_table()

for f,g in bm.face_attributes["parent"].items():
    if f == g:
        continue
    # v, w = dict_tmp[f], dict_tmp[g]
    v, w = tmp_mesh.verts[f.index],tmp_mesh.verts[g.index]
    tmp_mesh.edges.new((v,w))

    # vi, vj, vk = g.verts[0], g.verts[1], g.verts[2]
    # vi, vj, vk = tmp_mesh.verts[vi.index],tmp_mesh.verts[vj.index], tmp_mesh.verts[vk.index]
    # try:
    #     tmp_mesh.faces.new((vi, vj, vk))
    # except ValueError:
    #     pass

tmp_mesh.to_mesh(mesh_new)

dual_cycles = get_dual_cycles(bm)

mesh_new = bpy.data.meshes.new("MyMesh")
obj_new = bpy.data.objects.new(f"{obj.name}_dual_generators", mesh_new)
bpy.context.collection.objects.link(obj_new)
tmp_mesh = bmesh.new()

for v in bm.verts:
    tmp_mesh.verts.new(v.co)
tmp_mesh.verts.ensure_lookup_table()
for cycle in dual_cycles:
    for he in cycle:
        vi, vj = he.verts[0], he.verts[1]
        vi, vj = tmp_mesh.verts[vi.index], tmp_mesh.verts[vj.index]
        try:
            tmp_mesh.edges.new((vi, vj))
        except ValueError:
            pass
    
tmp_mesh.to_mesh(mesh_new)



In [56]:
tc.build_vector_b(directional_holonomies)

(1999, 1)
[[0.00540054]
 [0.00801942]
 [0.00940796]
 ...
 [0.01095851]
 [0.16059976]
 [0.08099934]]


In [51]:
directional_holonomies

[0.10839531123809006,
 0.12817528705273473,
 0.09619696886882934,
 0.3722054487040115,
 0.3939939052626931,
 0.26711020375902794,
 0.0430438735792742,
 1.1966041309486257,
 0.008154207512803414,
 0.0312568869513584,
 0.010958511282690995,
 0.1605997570626757,
 0.08099933788975255]

In [28]:
A = bm._build_d0().T
d1 = bm._build_d1()
# KK = -np.array([calculate_holonomy(bm, i) for i in range(len(bm.verts))])
KK = bm.vertex_attributes["angle_defect"]

b = KK.copy()
b[0] = b[0] - 2*np.pi * 2

AAT = A @ A.T
y = spsolve(AAT, -b)
x = A.T @ y

print(np.max(np.abs(A@x+b)))


2.398081733190338e-13


In [54]:
K

array([0.00540054, 0.00801942, 0.00940796, ..., 0.00484327, 0.00484273,
       0.00950955])

In [55]:
KK

array([-0.00540054, -0.00801942, -0.00940796, ..., -0.00484327,
       -0.00484273, -0.00950955])

In [57]:
np.max(np.abs(KK + K))

3.723128472188364e-10

In [59]:
if 'vector_field' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["vector_field"])
    mesh.attributes.remove(mesh.attributes["e1"])



attr = mesh.attributes.new(name="vector_field", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', vector_field.flatten())

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', bm.face_attributes["e1"].flatten())

In [82]:
mesh.attributes

ReferenceError: StructRNA of type Mesh has been removed